We use a dataset included in scikit-learn. 
This returns a dataset as dictionary which contains the description:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

dataset = load_diabetes()

X = pd.DataFrame(data=dataset['data'],columns=dataset['feature_names'])

# Again, best to scale the input variables
X = StandardScaler().fit_transform(X)

y = pd.DataFrame(data=dataset['target'],columns=['progression'])

In [2]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

svr = SVR(gamma='auto')
svr.fit(X_train,y_train.values.ravel())
prediction = svr.predict(X_test)
print('RMSE:', np.sqrt(mse(y_test,prediction)))

RMSE: 68.56822550246201


In [3]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':['linear','poly','rbf'],'C':[0.2,0.5,1.0]}

grid_search = GridSearchCV(SVR(gamma='auto'), parameters, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train.values.ravel())

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean RMSE (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/- %0.03f) for %r"
          # The MSE is return as a negative, so we multiple it with -1 before squaring it
          % (np.sqrt(-1*mean), np.sqrt(std), params))

Mean RMSE (+/- standard deviation), for parameters
58.366 (+/- 21.723) for {'C': 0.2, 'kernel': 'linear'}
75.202 (+/- 23.288) for {'C': 0.2, 'kernel': 'poly'}
77.576 (+/- 23.992) for {'C': 0.2, 'kernel': 'rbf'}
55.169 (+/- 22.722) for {'C': 0.5, 'kernel': 'linear'}
71.881 (+/- 23.362) for {'C': 0.5, 'kernel': 'poly'}
75.837 (+/- 23.567) for {'C': 0.5, 'kernel': 'rbf'}
54.365 (+/- 23.128) for {'C': 1.0, 'kernel': 'linear'}
69.069 (+/- 23.466) for {'C': 1.0, 'kernel': 'poly'}
73.398 (+/- 23.656) for {'C': 1.0, 'kernel': 'rbf'}


It seems that, again, the linear kernel is working best with the cost parameter only influencing the result slightly.
The confidence intervals are quite wide, however, so the results might not be very reliable.